In [1]:
import pandas as pd
import os

In [62]:
cvs_fp = r'C:\Users\iya\Dev\CVM\CVS\2022'
cvs_tour_fn = r'sandag_cvs_tour.xlsx'
tnc_tour_fn = r'sandag_tnc_tour.xlsx'
cvs_df_fn = r'cv_20221102_sandag_draftfinal.xlsx'
db_fn = r'SANDAG 2022 CV DataBase & Dictionaires_03_03_2023.xlsx'

pd.options.display.float_format = '{:20,.2f}'.format

In [3]:
establishment_data = pd.read_excel(os.path.join(cvs_fp, db_fn), sheet_name='Establishment Data')
trip_data = pd.read_excel(os.path.join(cvs_fp, db_fn), sheet_name='Trip Data')
tour_data = pd.read_excel(os.path.join(cvs_fp, cvs_tour_fn), sheet_name='sandag_cvs_tour')



In [4]:
employees = establishment_data[['base_location_Industry Group', 'no_of_emp_work']]
employees = employees.groupby('base_location_Industry Group').sum()
employees

,no_of_emp_work
base_location_Industry Group,
1,1114
2,6618
3,1025
4,6040
5,2852
6,3785
7,2133
8,6264
9,8220


In [5]:
trip = trip_data.merge(establishment_data[
    ['company_id', 'base_location_Industry Group']
    ], on='company_id', how='left')
gc = trip[
    ['base_location_Industry Group', 'Most Likely Estimate Weight Factor']
    ].groupby('base_location_Industry Group').sum()
gc

,Most Likely Estimate Weight Factor
base_location_Industry Group,
1,12155.782078
2,120978.735540
3,21123.987943
4,94282.137993
5,79692.935931
6,200942.196121
7,102064.118758
8,262026.912698
9,538669.756411


In [6]:
tour_data.columns

Index(['company_id', 'driver_id', 'trip_number', 'vehicle_id',
       'STOP_BASE_NOTBASE', 'TOUR_NUM', 'STOP_SEQ', 'trip_load_status',
       'trip_load_weight', 'activity_type', 'activity_type_other', 'placetype',
       'placetype_other', 'location_placename', 'location_address',
       'location_city', 'location_state', 'location_zip', 'cargo_pickup',
       'cargo_other_pickup', 'cargo_delivery', 'cargo_other_delivery',
       'cargo_pu_weight', 'cargo_do_weight', 'travel_date', 'arrival_time',
       'departure_time', 'used_other_vehicle', 'participation_type',
       'Lower.Estimate.Weight.Factor', 'Most.Likely.Estimate.Weight.Factor',
       'Upper.Estimate.Weight.Factor', 'TRAVEL_DISTANCE', 'TRAVEL_MINUTES',
       'stop_num', 'VehClass', 'StopPurpose', 'ActivityName', 'SORT_ORIG',
       'sort_clo', 'StopDuration', 'VehClassLMH', 'IndName', 'DistBin',
       'TimeBin', 'DistRound', 'TimeRound', 'OBJECTID', 'TAZ', 'Shape_Leng',
       'Shape_Area', 'geometry', 'TOUR_STOP_TAZID'

In [15]:

tour = tour_data[['company_id', 'TOUR_NUM']].groupby('company_id').nunique().reset_index()
tour = tour.merge(establishment_data[
    ['company_id', 'base_location_Industry Group']
    ], on='company_id', how='left')
tour
#tour[['TOUR_NUM', 'base_location_Industry Group']].groupby('base_location_Industry Group').sum()

,company_id,TOUR_NUM,base_location_Industry Group
0,100012,2,9
1,100021,1,1
2,100023,1,4
3,100039,1,1
4,100050,1,1
...,...,...,...
451,600041,2,8
452,600051,2,4
453,600056,1,6
454,600145,3,10


In [18]:
tour_data[['company_id','Most.Likely.Estimate.Weight.Factor']].groupby('company_id').nunique()

,Most.Likely.Estimate.Weight.Factor
company_id,
100012,1
100021,1
100023,1
100039,1
100050,1
...,...
600041,2
600051,1
600056,1


In [23]:
tour_id = []
counter = 0
start, end = tour_data['TOUR_START'], tour_data['TOUR_END']

for se in list(zip(start, end)):
    if se == (True, False):
        counter += 1
    tour_id.append(counter)

In [26]:
tour_data['tour_id'] = tour_id

In [41]:
tour_data = tour_data.merge(establishment_data[
    ['company_id', 'base_location_Industry Group']
    ], on='company_id', how='left')

In [54]:
tour_1 = tour_data[['Most.Likely.Estimate.Weight.Factor', 'tour_id']].groupby('tour_id').mean().reset_index()
tour_1 = tour_1.merge(
    tour_data[['base_location_Industry Group', 'tour_id']].groupby('tour_id').mean(), 
    on='tour_id',
    how='left',
    validate='one_to_many'
    )
tour_1[
    ['Most.Likely.Estimate.Weight.Factor', 'base_location_Industry Group']
].groupby('base_location_Industry Group').sum().reset_index()

,base_location_Industry Group,Most.Likely.Estimate.Weight.Factor
0,1.0,2386.821215
1,2.0,22801.712597
2,3.0,5409.176934
3,4.0,18768.581199
4,5.0,13948.570087
5,6.0,42326.071740
6,7.0,12798.121199
7,8.0,57081.146077
8,9.0,84795.471911
9,10.0,51360.682482


In [64]:
vmt = tour_data[
    ['Most.Likely.Estimate.Weight.Factor', 'base_location_Industry Group', 'TRAVEL_DISTANCE']
].dropna(how='any')
vmt['weighted_distance'] = vmt['Most.Likely.Estimate.Weight.Factor'] * vmt.TRAVEL_DISTANCE
vmt[['weighted_distance', 'base_location_Industry Group']].groupby('base_location_Industry Group').sum().reset_index()

,base_location_Industry Group,weighted_distance
0,1,"94,294.62"
1,2,"1,455,627.51"
2,3,"94,476.61"
3,4,"662,266.36"
4,5,"1,054,351.58"
5,6,"1,996,970.71"
6,7,"753,000.36"
7,8,"2,000,252.90"
8,9,"2,830,538.79"
9,10,"3,226,709.41"


In [55]:
tour_data.columns

Index(['company_id', 'driver_id', 'trip_number', 'vehicle_id',
       'STOP_BASE_NOTBASE', 'TOUR_NUM', 'STOP_SEQ', 'trip_load_status',
       'trip_load_weight', 'activity_type', 'activity_type_other', 'placetype',
       'placetype_other', 'location_placename', 'location_address',
       'location_city', 'location_state', 'location_zip', 'cargo_pickup',
       'cargo_other_pickup', 'cargo_delivery', 'cargo_other_delivery',
       'cargo_pu_weight', 'cargo_do_weight', 'travel_date', 'arrival_time',
       'departure_time', 'used_other_vehicle', 'participation_type',
       'Lower.Estimate.Weight.Factor', 'Most.Likely.Estimate.Weight.Factor',
       'Upper.Estimate.Weight.Factor', 'TRAVEL_DISTANCE', 'TRAVEL_MINUTES',
       'stop_num', 'VehClass', 'StopPurpose', 'ActivityName', 'SORT_ORIG',
       'sort_clo', 'StopDuration', 'VehClassLMH', 'IndName', 'DistBin',
       'TimeBin', 'DistRound', 'TimeRound', 'OBJECTID', 'TAZ', 'Shape_Leng',
       'Shape_Area', 'geometry', 'TOUR_STOP_TAZID'

In [72]:
light, med, hvy = list(range(1, 4)), list(range(5, 8)), [4, 8]
def classing(num):
    if num in light: return 'light'
    elif num in med: return 'med'
    else: return 'hvy'
tour_data['VehClass_name'] = tour_data.VehClass.apply(lambda x: classing(x))

vmt = tour_data[
    ['Most.Likely.Estimate.Weight.Factor', 'base_location_Industry Group', 'TRAVEL_DISTANCE', 'VehClass_name']
].dropna(how='any')
vmt = vmt[vmt.VehClass_name != 'light']
vmt['weighted_distance'] = vmt['Most.Likely.Estimate.Weight.Factor'] * vmt.TRAVEL_DISTANCE
vmt[['weighted_distance', 'base_location_Industry Group']].groupby('base_location_Industry Group').sum().reset_index()


,base_location_Industry Group,weighted_distance
0,1,"75,846.32"
1,2,"1,252,495.63"
2,3,"53,948.85"
3,4,"381,875.91"
4,5,"412,153.19"
5,6,"1,007,088.57"
6,7,"367,360.34"
7,8,"474,896.71"
8,9,"1,205,466.61"
9,10,"1,347,647.49"
